# What to do with inline code blocks in Markdown

We've wanted to experiment with the IPython inspector for a while.  Rather than
change the way we predict objects, this post uses the default inspection view to augmented rendering of Markdown forward code.



Recently, some posts have been introduced as changes to `pidgin` which provides some literate computing extensions for the Jupyter notebook.  It effectively combines the previous posts on [String Transformers](2018-06-19-String-Node-Transformer.ipynb), [Markdown Code Cells](2018-07-06-Markdown-code-cells.ipynb), and [Jinja Templated Cells](2018-07-07-A-Jinja2-Templating-Transformer.md.ipynb).

In [1]:
    from . import (
        __String_Node_Transformer, __Markdown_code_cells, 
        __Jinja2_Templating_Transformer)

In [2]:
    from CommonMark import Parser
    from pidgin.markdown import MarkdownTransformer, MarkdownImporter, CodeRenderer
    from CommonMark.render.html import HtmlRenderer
    import ast
    from tqdm import tqdm
    from IPython.display import display, HTML
    from IPython import get_ipython
    from IPython.core.oinspect import Inspector
    from IPython.utils.capture import capture_output
    inspector = Inspector(scheme='NoColor')

* Assign a special css styles to the rendered inline code blocks.

In [3]:
    style = """
    <style>
    .inspectable {display: inline;}
    .inspectable code {
        display: inline-block;
        text-decoration: underline;}
    .inspectable:hover code {text-decoration: overline;}
    .inspectable .inspect {
        display: none;
        border-style: solid;
        border-width: 5px;}
    .inspectable:hover .inspect {display: block;}</style>"""
    display(HTML(style))

* A custom renderer for inline code cells that evaluates code.

In [4]:
    class IMarkdownRenderer(HtmlRenderer):
        def code(self, node, entering):
            with capture_output(stderr=True, stdout=False) as out:
                module = ast.parse(node.literal)
                for object in tqdm(module.body, node.literal):
                    if object is module.body[-1] and isinstance(module.body[-1], ast.Expr):
                        code = compile(ast.Expression(object.value), '<inspected>', 'eval')
                    else:
                        code = compile(ast.Module([object]), '<inspected>', 'exec')
                    object = eval(code, get_ipython().user_ns)
                if object is not None:
                    display(object)

            if out.outputs and 'text/html' in  out.outputs[0].data:
                overlay = out.outputs[0].data['text/html']
            elif object is not None:
                overlay = inspector._get_info(object, node.literal)['text/html']
            else: 
                overlay = ""

            begin, end = self.buf.rsplit('>', 1)

            self.buf = begin + ' style="display: inline-block">' + end
            self.tag('div', (('class', 'inspectable'),))
            super().code(node, entering)
            self.tag('div', (('class', 'inspect'),))
            self.tag('pre'), self.tag('code')
            self.out(out.stderr.splitlines()[-1])
            self.tag('/code'), self.tag('/pre')

            if overlay: self.lit(overlay), self.tag('br', selfclosing=True)
            self.tag('/div'), self.tag('/div')
            return

In [5]:
    class InteractiveRenderer(CodeRenderer):
        def repr(self, str): return HTML(IMarkdownRenderer().render(Parser().parse(str)))


* Create a transformer and importer

In [6]:
    class IMarkdownTransformer(MarkdownTransformer): renderer = staticmethod(InteractiveRenderer())

    class IMarkdownImporter(MarkdownImporter):
        extensions = '.i.md.ipynb',
        renderer = IMarkdownTransformer.renderer

In [7]:
    def load_ipython_extension(ip=None):
        ip = ip or get_ipython()
        unload_ipython_extension(ip)
        ip.input_transformer_manager.physical_line_transforms.insert(0, IMarkdownTransformer())            
    def unload_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager.physical_line_transforms = list(
            object for object in ip.input_transformer_manager.physical_line_transforms
            if not isinstance(object, (MarkdownTransformer, IMarkdownTransformer))
        )

# Results

In [8]:
    if __name__ == '__main__': load_ipython_extension()

In [9]:
## What is special now; `import pandas, pidgin`?
---

The `pidgin.markdown` module allow code cells to be markdown and the block code is concatenated into a single code execution.
The inline code elements are meaningless by these conventions.

> It has been nagging me for a while what to do with inline code in Markdown.  

## Interactive Markdown

In interactive markdown, inline code cells have augmented views.  Like the author would share what 
the current state of the `globals().keys()` are.  If the notebook will restart and run all then these
outputs can be encoded into documents.

Usually there is and data.  When there is data with `import pandas as pd;pd`
and create a `pd.DataFrame`.  The quickest way to do so is with the `pd.util.testing` module; 
`df = pd.util.testing.makeDataFrame(); df` and has the following properties. 

    df.describe()

,A,B,C,D
UHPvExxqlh,-0.476295,-0.130005,0.066832,0.017031
I51XYsQZBF,-0.551801,-0.254836,1.085106,1.591310
62Fx8izCuh,-0.140545,0.991642,-0.594077,-0.024814
Ezc13mXfNJ,0.915035,-3.392788,0.913615,0.040751
VvRKB9JZ2F,0.759155,0.956740,-0.939380,-0.379307
udCi4LIPlU,-0.559881,0.975863,-1.404088,-0.420175
71n7xBWAiA,0.126133,1.442253,0.877734,-0.310031
fdQRx44i4k,0.255769,1.223504,0.886755,1.509925
7Il3tn4yDB,-1.220453,-0.314572,0.707834,-2.347341
xX5MXrN13X,0.471530,1.235290,-0.587592,0.586608


,A,B,C,D
count,30.000000,30.000000,30.000000,30.000000
mean,0.179578,0.146087,-0.223768,0.102723
std,0.815256,1.264347,1.001570,0.815346
min,-1.599355,-3.392788,-2.220132,-2.347341
25%,-0.392357,-0.423650,-0.937945,-0.343664
50%,0.202744,0.148306,-0.431079,0.059032
75%,0.795423,0.971082,0.638918,0.680518
max,1.566004,3.609693,1.711016,1.591310
